# TP5

## Imports

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import numpy as np
import matplotlib.pyplot as plt

## Chargement des données

Les fonctions pour charger les bases Movie Lens 100k et Movie Lens 1M.
On récupère un dictionnaire pour les scores et un dictionnaire pour les dates.
Le première index de ces dictionnaires est l'identifiant de l'utilisateur, et le second les films notés.

In [ ]:
def loadMovieLens(path='./data100k'):
    # Get movie titles
    movies={}
    for line in open(path+'/u.item'):
        (id,title)=line.split('|')[0:2]
        movies[id]=title
    # Load data
    prefs={} # Un dictionnaire User > Item > Rating
    times={} # Un dictionnaire User > Item > Timestamps
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
        times.setdefault(user,{})
        times[user][movies[movieid]]=float(ts)
    return prefs, times

In [ ]:
def loadMovieLens1M(path='./data1m'):
    # Get movie titles
    movies={}
    for line in open(path+'/movies.dat'):
        id,title=line.split('::')[0:2]
        movies[id]=title
    # Load data
    prefs={}
    times={}
    for line in open(path+'/ratings.dat'):
        (user,movieid,rating,ts)=line.split('::')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
        times.setdefault(user,{})
        times[user][movies[movieid]]=float(ts)
    return prefs, times

## Représentations des données

Les matrices des scores Utilisateurs/Films sont des matrices de grandes dimensions mais sparses.
Afin de les manipuler efficacement, on emploiera 3 représentations différentes en même temps:
- Le dictionnaire des scores par utilisateurs: User > Item > Value
- Le dictionnaire des scores par films: Item > User > Value 
- La liste des triplets [User, Item, Value]

In [ ]:
# Recupère une représentation des données sous la forme triplets [user, item, value] a partir d'un dictionnaire [User > item > value]
def getCouplesUsersItems(data):
    couples = []
    for u in data.keys():
        for i in data[u].keys():
            couples.append([u,i,data[u][i]])
    return couples

# Construit le dictionnaire des utilisateurs a partir des triplets [user, item, note]
def buildUsersDict(couples):
    dicUsers = {}
    for c in couples:
        if not c[0] in dicUsers.keys():
            dicUsers[c[0]] = {}
        dicUsers[c[0]][c[1]] = float(c[2])
    return dicUsers

# Construit le dictionnaire des objets a partir des triplets [user, item, note]
def buildItemsDict(couples):
    dicItems = {}
    for c in couples:
        if not c[1] in dicItems:
            dicItems[c[1]] = {}
        dicItems[c[1]][c[0]] = float(c[2])
    return dicItems

## Données de temps

Afin d'exploiter les données temporelles, on discrétise le temps en bins et on construit un vecteur qui a chaque triplets [user, item, value] associe le bins temporel correspondant.

In [ ]:
def getTimeBins(couples, timedic, nbins):
    timestamps = np.zeros(len(couples))
    for i,c in enumerate(couples):
        timestamps[i] = timedic[c[0]][c[1]]
    time_bins = np.linspace(np.min(timestamps), np.max(timestamps), nbins+1)
    times = np.zeros(len(couples),int)
    for i in xrange(1,len(time_bins)):
        times = times + (timestamps > time_bins[i])
    return times

## Séparation des données en Train / Test

Pour pouvoir séparer les données en ensembles de Train et de Test, on utilisera la liste des triplets [User, Item, Scores].

In [ ]:
# Split l'ensemble des triplets [user, item, note] en testProp% données de test et (1 - testProp) données de train
def splitTrainTest(couples,testProp):
    perm = np.random.permutation(couples)
    splitIndex = int(testProp * len(couples))
    return perm[splitIndex:], perm[:splitIndex]

# Modèles

On implémente ici les différents modèles. Les baselines prédisent simplement la note moyenne pour un utilisateur (ou pour un film) donné. Les modèles de factorisation matricielles tentent d'approximer les valeurs connues de la matrice des scores par un produit de deux matrices de dimensions inférieurs.

## Factorisation matricielle sans biais

On calcule les deux matrices P et Q tel que pour les exemples connus, PQ ~= X, où X est la matrice des scores.
Pour prédire, il suffit alors de lire dans la matrice PQ les nouveaux exemples.

In [2]:
class matrixFactorisation():
    def __init__(self, k, lambd=0.2, eps=1e-5, maxIter=2000, alternate=0):
        self.k = k
        self.lambd = lambd
        self.eps = eps
        self.maxIter = maxIter
        self.alternate = alternate #Pour l'optimisation alternée: 0 si non.
    def fit(self, dataUsers, dataItems, couples):
        self.p = {}
        self.q = {}
        self.loss = []
        #Choix du paramètre a optimisé en cas d'optimisation alternée
        optimP = True
        optimQ = (self.alternate == 0)
        for i in xrange(self.maxIter):
            loss = 0
            for j in xrange(len(couples)):
                #choix d'une entrée aléatoire
                r = np.random.randint(len(couples)) 
                user = couples[r][0]
                item = couples[r][1]
                # initialisation des nouveaux vecteurs p et q
                if not user in self.p:
                    self.p[user] = np.random.rand(1,self.k)
                if not item in self.q:
                    self.q[item] = np.random.rand(self.k,1)
                # Descente de gradient
                tmp = dataUsers[user][item] - self.p[user].dot(self.q[item])[0][0]
                if (optimP):
                    self.p[user] = (1 - self.lambd * self.eps) * self.p[user] + self.eps * 2 * tmp * self.q[item].transpose()
                if (optimQ):
                    self.q[item] = (1 - self.lambd * self.eps) * self.q[item] + self.eps * 2 * tmp * self.p[user].transpose()
                loss = loss + tmp*tmp #(Sans le terme de régularisation)
            self.loss.append(loss)
            # Optimisation alternée
            if (self.alternate != 0):
                if (i % self.alternate == 0):
                    optimP = optimQ
                    optimQ = 1 - optimQ
                    print i, loss / len(couples)
            else:
                if (i % 100 == 0):
                    print i, loss / len(couples)
    def predict(self, couplesTest):
        pred = np.zeros(len(couplesTest))
        for ind,c in enumerate(couplesTest):
            pred[ind] = self.p[c[0]].dot(self.q[c[1]])[0][0]
        return pred

# Tests les données Movie Lens 100k

Les données Movie Lens 100k comprennent 100 000 scores données par 1000 utilisateurs sur 1700 films.

## Préparation des données

On extrait aléatoirement une portion (20%) des données pour constituer la base de test, et le reste sera utilisé en apprentissage.

Comme on ne souhaite ne pas évaluer les objets et les utilisateurs qui n'ont jamais été rencontré en apprentissage, on retire les couples correspondants de l'ensemble de test.

Reste ensuite à reconstruire les deux dictionnaires a partir de ces liste de couples.

In [ ]:
# Chargement
data, timestamps = loadMovieLens()

# Récupérer la représentation en liste de triplets
couples = getCouplesUsersItems(data)

# La séparer en ensemble d'apprentissage et de test
trainCouples, testCouples = splitTrainTest(couples,.20)

# Reconstruire les dictionnaires pour l'ensemble d'apprentissage
trainUsers = buildUsersDict(trainCouples)
trainItems = buildItemsDict(trainCouples)

# Supprimer de l'ensemble de test les éléments inconnus en apprentissage
toDel = []
for i,c in enumerate(testCouples):
    if not c[0] in trainUsers:
        toDel.append(i)
    elif not c[1] in trainItems:
        toDel.append(i)
testCouples = np.delete(testCouples, toDel, 0)

# Reconstruire les dictionnaires pour l'ensemble de test
testUsers  = buildUsersDict(testCouples)
testItems  = buildItemsDict(testCouples)

# taille des données
#print len(trainUsers), len(testUsers)
#print len(trainItems), len(testItems)

## Factorisation matricielle sans biais

Après apprentissage, la factorisation matricielle donne une erreur moyenne en test de 0.9
Il est meilleur que les baselines de 0.1 point.
On note que le loss en apprentissage est de 0.84. 
Il est possible que l'on obtienne de meilleurs score de généralisation en test en augmentant la régularisation mais au vu du score actuel en apprentissage, le gain devrait rester assez faible.

In [ ]:
model3 = matrixFactorisation(10, alternate=0)
model3.fit(trainUsers, trainItems, trainCouples)

In [ ]:
plt.figure()
plt.plot(model3.loss)
plt.show()

In [ ]:
pred = model3.predict(testCouples)
print "Erreur de test:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

# Experiences sur les données Movie Lens 1M

Le dataset Movie Lens 1M contient 1 million d'entrées, données par 6000 utiilisateurs sur 4000 films.

Une remarque que l'on peut déjà faire est que la matrice des scores est "moins sparse" que celle issue de la base 100k.

En effet, pour la base 100k on a 100000 notes pour une matrice 1000x1700, soit un remplissage de 17% de la matrice.

Pour la base 1M, on a 1 000 000 de notes pour une matrice 6000x4000, soit 24% de remplissage.

## Préparation des données

In [ ]:
# Chargement
data, timestamps = loadMovieLens1M()

# Récupérer la représentation en liste de triplets
couples = getCouplesUsersItems(data)

# Séparer en ensemble d'apprentissage et de test
trainCouples, testCouples = splitTrainTest(couples,.20)

# Reconstruire les dictionnaires pour l'ensemble d'apprentissage
trainUsers = buildUsersDict(trainCouples)
trainItems = buildItemsDict(trainCouples)

# Supprimer de l'ensemble de test les éléments inconnus en apprentissage
toDel = []
for i,c in enumerate(testCouples):
    if not c[0] in trainUsers:
        toDel.append(i)
    elif not c[1] in trainItems:
        toDel.append(i)
testCouples = np.delete(testCouples, toDel, 0)

# Reconstruire les dictionnaires pour l'ensemble de test
testUsers  = buildUsersDict(testCouples)
testItems  = buildItemsDict(testCouples)

# taille des données
#print len(trainUsers), len(testUsers)
#print len(trainItems), len(testItems)

## Factorisation Matricielle

Avec un score en apprentissage de 0.82 et de généralisation de 0.85, on peut estimer que le paramètre de régularisation choisi (lambda = 0.2) est raisonnable pour ce problème. La factorisation matricielle est aussi meilleure que celle de la base 100k, probablement parceque la matrice d'apprentissage est moins sparse.

In [ ]:
model8 = matrixFactorisation(10, alternate=0, maxIter=1000)
model8.fit(trainUsers, trainItems, trainCouples)

In [ ]:
plt.figure()
plt.plot(model8.loss)
plt.show()

In [ ]:
pred = model8.predict(testCouples)
print ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

# Conclusion

Dans ce travail, nous avons implémenté des modèles de filtrage collaboratif que nous avons ensuite chercher à évaluer.
Si on a observé qu'un modèle simple de factorisation matricielle pouvait obtenir des résultats concluants, meilleurs qu'une baseline naïve, nous n'avons pu démontré d'intérêt pratiques des modèles avec biais.

Cependant, nous avons observés que si nos modèles sans biais avaient des scores similaires en apprentissage qu'en généralisation, ce n'était pas le cas de nos modèles avec biais qui obtiennent de bien meilleurs scores en apprentissage qui ne se traduisent pas en test. On peut en déduire que nous n'avons pas déterminé les bons hyperparamètres pour permettre une bonne généralisation, et que vraisemblablement on peut encore augmenter le score en généralisation de nos modèles avec biais.

Enfin, nous n'avons eu le temps ni d'évaluer les modèles avec biais temporel, ni l'influence de la dimension de factorisation matricielle, que l'on peut aussi voir comme une forme de régularisation.

In [79]:
sigma = np.array([1,2,3,10],float)
sigma = sigma.reshape(4,1)
print sigma

[[  1.]
 [  2.]
 [  3.]
 [ 10.]]


In [80]:
A = np.array([[1,2,3],[4,5,6],[1,3,2],[4,5,6]], float)
print A

[[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 1.  3.  2.]
 [ 4.  5.  6.]]


In [82]:
norm = np.sum(A**2,1)
norm = np.reshape(norm, (1, np.shape(norm)[0]))
distance = norm.T + norm - 2 * A.dot(A.T)
print distance

#for i in xrange(4):
#    for j in xrange(4):
#        print (((A[i] - A[j])**2).sum() - distance[i,j])

[[  0.  27.   2.  27.]
 [ 27.   0.  29.   0.]
 [  2.  29.   0.  29.]
 [ 27.   0.  29.   0.]]


In [85]:
p = np.exp(- distance / (2. * (sigma**2)))
#print p
#print (p - np.eye(4)).sum(1).reshape(4,1)

pisj = p / (p - np.eye(4)).sum(1).reshape(4,1)

print pisj

#for i in xrange(4):
#    s = 0
#    for k in xrange(4):
#        if i != k:
#            s += np.exp(-distance[i,k]/(2*(sigma[i,0]**2)))
#    for j in xrange(4):
#        print pisj[i,j] - (np.exp(-distance[i,j]/(2*(sigma[i,0]**2)))/s)

[[  2.71826157e+00   3.72662540e-06   9.99992547e-01   3.72662540e-06]
 [  3.22548551e-02   9.42625039e-01   2.51201064e-02   9.42625039e-01]
 [  6.91438454e-01   1.54280773e-01   7.72695657e-01   1.54280773e-01]
 [  3.19021333e-01   3.65131650e-01   3.15847017e-01   3.65131650e-01]]


In [86]:
entropy = -(pisj * np.log2(pisj)).sum(0)
print entropy

[-2.86788526  1.02714144  0.94614824  1.02714144]


  $-\sum_j p_{j|i} log_2(p_{j|i})$ 
  
$= -\sum_j p_{j|i} log_2(\frac{exp(-d(x_i, x_j)^2 / (2\sigma^2_i)}{\sum_{k\ne i}exp(-d(x_i, x_k)^2 / (2\sigma^2_i)} )$

$= -\sum_j p_{j|i} (log_2(exp(-d(x_i, x_j)^2 / (2\sigma^2_i)) - log_2(\sum_{k\ne i}exp(-d(x_i, x_k)^2 / (2\sigma^2_i))) )$

$= -\sum_j p_{j|i} (exp(-d(x_i, x_j)^2 / (2\sigma^2_i))/ln(2) - log_2(\sum_{k\ne i}exp(-d(x_i, x_k)^2 / (2\sigma^2_i))) )$

In [77]:
def perp(data, sigma):
    norm = np.sum(data**2,1)
    norm = np.reshape(norm, (1, np.shape(norm)[0]))
    distance = norm.T + norm - 2 * data.dot(data.T)
    p = np.exp(- distance / (2. * (sigma**2)))
    pisj = p / (p - np.eye(10)).sum(1).reshape(10,1)
    entropy = -(pisj * np.log2(pisj)).sum(0)
    return entropy
print perp(A, sigma)

[              nan               nan -6738487.12685308 -6738487.12685308
 -6738487.12685308 -6738487.12685308 -6738487.12685308 -6738487.12685308
               nan               nan]


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in multiply


In [78]:
A = np.array(range(30),float).reshape(10,3)
sigma = np.ones((10,1))
print perp(A,sigma)
sigma = np.ones((10,1))*2
print perp(A,sigma)
sigma = np.ones((10,1))*3
print perp(A,sigma)
sigma = np.ones((10,1))*4
print perp(A,sigma)
sigma = np.ones((10,1))*5
print perp(A,sigma)
sigma = np.ones((10,1))*6
print perp(A,sigma)
sigma = np.ones((10,1))*7
print perp(A,sigma)
sigma = np.ones((10,1))*8
print perp(A,sigma)
sigma = np.ones((10,1))*9
print perp(A,sigma)
sigma = np.ones((10,1))*10
print perp(A,sigma)
sigma = np.ones((10,1))*1500
print perp(A,sigma)

print np.log2(5)

[              nan               nan -6738487.12685308 -6738487.12685308
 -6738487.12685308 -6738487.12685308 -6738487.12685308 -6738487.12685308
               nan               nan]
[-141.78812924  -56.03384214  -55.53175552  -55.53202366  -55.53202366
  -55.53202366  -55.53202366  -55.53175552  -56.03384214 -141.78812924]
[-8.97875163 -2.0167422  -1.42803277 -1.4568077  -1.45704031 -1.45704031
 -1.4568077  -1.42803277 -2.0167422  -8.97875163]
[-1.6884833   0.62029314  1.36670578  1.28760176  1.27851107  1.27851107
  1.28760176  1.36670578  0.62029314 -1.6884833 ]
[ 0.14538689  1.37705668  2.15843492  2.12703618  2.08832637  2.08832637
  2.12703618  2.15843492  1.37705668  0.14538689]
[ 0.92808892  1.78169703  2.50984921  2.59454227  2.53968208  2.53968208
  2.59454227  2.50984921  1.78169703  0.92808892]
[ 1.35918804  2.0472798   2.70321051  2.89850664  2.87310155  2.87310155
  2.89850664  2.70321051  2.0472798   1.35918804]
[ 1.63602334  2.23862531  2.83197707  3.10752257  3.145329

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in multiply


In [60]:
print perp(A, sigma)

[-2.86788526  1.02714144  0.94614824  1.02714144]


In [ ]:
np.sum(p - np.eye(np.shape(p)[0]),0)

In [ ]:
ps=p / np.sum(p - np.eye(np.shape(p)[0]),0)
print ps

In [ ]:
(ps + ps.T) / (2*np.shape(ps)[0])

In [411]:
data = np.array([[1,2,3],[1,4,3],[7,8,9],[10,11,12]], float)
print data

[[  1.   2.   3.]
 [  1.   4.   3.]
 [  7.   8.   9.]
 [ 10.  11.  12.]]


In [383]:
#y = np.random.randn(np.shape(data)[0], 2) * 1e-4
y = np.array([[1,2],[2,3],[4,5],[0,1]], float)
print y

[[ 1.  2.]
 [ 2.  3.]
 [ 4.  5.]
 [ 0.  1.]]


In [384]:
pjsi = np.zeros((np.shape(data)[0], np.shape(data)[0]))
print pjsi

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [385]:
sigma = np.ones(np.shape(data)[0]) * 10
print sigma

[ 10.  10.  10.  10.]


In [386]:
normx = np.sum((data**2),1)
print normx
print ""
normx = np.reshape(normx, (1, np.shape(normx)[0]))
print normx
print ""
distancex = normx + normx.T - 2 * data.dot(data.T)
print repnormx + repnormx.T
print 2 * data.dot(data.T)
print distancex

[  14.   26.  194.  365.]

[[  14.   26.  194.  365.]]

[[ 28  40 208 379]
 [ 40  52 220 391]
 [208 220 388 559]
 [379 391 559 730]]
[[  28.   36.  100.  136.]
 [  36.   52.  132.  180.]
 [ 100.  132.  388.  532.]
 [ 136.  180.  532.  730.]]
[[   0.    4.  108.  243.]
 [   4.    0.   88.  211.]
 [ 108.   88.    0.   27.]
 [ 243.  211.   27.    0.]]


In [387]:
pjsi = np.exp(-distancex / (2. * sigma))
print pjsi

[[  1.00000000e+00   8.18730753e-01   4.51658094e-03   5.28837258e-06]
 [  8.18730753e-01   1.00000000e+00   1.22773399e-02   2.61934809e-05]
 [  4.51658094e-03   1.22773399e-02   1.00000000e+00   2.59240261e-01]
 [  5.28837258e-06   2.61934809e-05   2.59240261e-01   1.00000000e+00]]


In [388]:
print pjsi - np.eye(np.shape(pjsi)[0])
print np.sum(pjsi - np.eye(np.shape(pjsi)[0]),0)
print np.sum(pjsi - np.eye(np.shape(pjsi)[0]),1)
pjsi2 = pjsi / np.sum(pjsi - np.eye(np.shape(pjsi)[0]),0)
print pjsi2

print pjsi[0] / np.sum(pjsi - np.eye(np.shape(pjsi)[0]),0)
print pjsi[1] / np.sum(pjsi - np.eye(np.shape(pjsi)[0]),0)
print pjsi[2] / np.sum(pjsi - np.eye(np.shape(pjsi)[0]),0)
print pjsi[3] / np.sum(pjsi - np.eye(np.shape(pjsi)[0]),0)

[[  0.00000000e+00   8.18730753e-01   4.51658094e-03   5.28837258e-06]
 [  8.18730753e-01   0.00000000e+00   1.22773399e-02   2.61934809e-05]
 [  4.51658094e-03   1.22773399e-02   0.00000000e+00   2.59240261e-01]
 [  5.28837258e-06   2.61934809e-05   2.59240261e-01   0.00000000e+00]]
[ 0.82325262  0.83103429  0.27603418  0.25927174]
[ 0.82325262  0.83103429  0.27603418  0.25927174]
[[  1.21469397e+00   9.85194915e-01   1.63623973e-02   2.03970264e-05]
 [  9.94507313e-01   1.20331979e+00   4.44776072e-02   1.01027133e-04]
 [  5.48626366e-03   1.47735660e-02   3.62273974e+00   9.99878576e-01]
 [  6.42375431e-06   3.15191338e-05   9.39159995e-01   3.85695715e+00]]
[  1.21469397e+00   9.85194915e-01   1.63623973e-02   2.03970264e-05]
[  9.94507313e-01   1.20331979e+00   4.44776072e-02   1.01027133e-04]
[ 0.00548626  0.01477357  3.62273974  0.99987858]
[  6.42375431e-06   3.15191338e-05   9.39159995e-01   3.85695715e+00]


In [408]:
#print pjsi
for j in xrange(4):
    for i in xrange(4):
        #print pjsi[i,j] -  np.exp(- distancex[i,j] / 20.)
        pass
    print ''

#print pjsi2.T
for j in xrange(4):
    s = 0
    for k in xrange(4):
        if k != j:
            s += pjsi[k,j]
    for i in xrange(4):
        res = np.exp(-distancex[i,j] / sigmai[i]) / s
        print pjsi[i,j] / s
        print pjsi[i,j] / np.sum(pjsi - np.eye(np.shape(pjsi)[0]),0)[j]
        print (pjsi / np.sum(pjsi - np.eye(np.shape(pjsi)[0]),0))[i,j]
        #print res - pjsi2[j,i]
    #print ''

#print np.sum(pjsi - np.eye(np.shape(pjsi)[0]),1)

i = 1
j = 3

print pjsi2[i,j]
print np.exp(- np.sum((data[i] - data[j])**2) / sigmai[i]) / (np.exp(- np.sum((data[0] - data[j])**2) / sigmai[i]) + np.exp(- np.sum((data[1] - data[j])**2) / sigmai[i]) + np.exp(- np.sum((data[2] - data[j])**2) / sigmai[i]))





1.21469397461
1.21469397461
1.21469397461
0.994507312589
0.994507312589
0.994507312589
0.00548626365681
0.00548626365681
0.00548626365681
6.42375431005e-06
6.42375431005e-06
6.42375431005e-06
0.985194914838
0.985194914838
0.985194914838
1.20331978631
1.20331978631
1.20331978631
0.0147735660286
0.0147735660286
0.0147735660286
3.15191338004e-05
3.15191338004e-05
3.15191338004e-05
0.0163623972879
0.0163623972879
0.0163623972879
0.0444776072178
0.0444776072178
0.0444776072178
3.62273974403
3.62273974403
3.62273974403
0.939159995494
0.939159995494
0.939159995494
2.03970264186e-05
2.03970264186e-05
2.03970264186e-05
0.000101027133213
0.000101027133213
0.000101027133213
0.99987857584
0.99987857584
0.99987857584
3.85695714604
3.85695714604
3.85695714604
0.000101027133213


KeyError: 1

In [222]:
np.shape(pjsi)
np.shape


(4, 4)

In [394]:
distancex

array([[   0.,    4.,  108.,  243.],
       [   4.,    0.,   88.,  211.],
       [ 108.,   88.,    0.,   27.],
       [ 243.,  211.,   27.,    0.]])

In [407]:
sigmai = np.array([1,2,4,10]).reshape(4,1)
print sigmai

[[ 1]
 [ 2]
 [ 4]
 [10]]
[2]


In [402]:
distancex / sigma**2
print pjsi / np.sum(pjsi - np.eye(np.shape(pjsi)[0]),0)



[[  1.21469397e+00   9.85194915e-01   1.63623973e-02   2.03970264e-05]
 [  9.94507313e-01   1.20331979e+00   4.44776072e-02   1.01027133e-04]
 [  5.48626366e-03   1.47735660e-02   3.62273974e+00   9.99878576e-01]
 [  6.42375431e-06   3.15191338e-05   9.39159995e-01   3.85695715e+00]]


In [3]:
class tSNE():
    def __init__(self,perp, nIter, lr, moment, dim=2):
        self.perp = perp # entre 5 et 50
        self.nIter = nIter
        self.lr = lr
        self.moment = moment
        self.dim = dim    
    def fit(self,data):
        nEx = np.shape(data)[0]
        # Matrice des distances de ||xi - xj||² #
        normx = np.sum(data**2,1)
        normx = np.reshape(normx, (1, nEx))
        distancex = normx + normx.T - 2 * data.dot(data.T)
        # Calcul des sigma ---------------------------------------------------------------#
        lperp = np.log2(self.perp)
        # initialisation bornes pour la recherche dichotomique #
        sup = np.ones((nEx,1)) * np.max(distancex)
        inf = np.zeros((nEx,1))
        self.sigma = (sup + inf) / 2.
        # recherche dichotomique #
        stop = False
        while not stop:
            # Calculer la matrice des p(i|j)
            self.pcond = np.exp(-distancex / (2. * (self.sigma**2)))
            self.pcond = self.pcond / np.sum(self.pcond - np.eye(nEx),1).reshape(nEx,1)
            # Calculer l'entropie de p(i|j)
            entropy = - np.sum(self.pcond * np.log2(self.pcond), 0)
            # Mise a jour des bornes
              # Si il faut augmenter sigma
            up = entropy < lperp 
            inf[up,0] = self.sigma[up,0]
              # Si il faut baisser sigma
            down = entropy > lperp 
            sup[down,0] = self.sigma[down,0]
            # Mise a jour de sigma et condition d'arrêt
            old = self.sigma
            self.sigma = ((sup + inf) / 2.)
            if np.max(np.abs(old - self.sigma)) < 1e-5:
                stop = True
                print np.exp(entropy)
                print self.sigma.T  
        #--------------------------------------------------------------------------#
        #initialiser y
        self.embeddings = np.zeros((self.nIter+2, nEx, self.dim))
        self.embeddings[1] = np.random.randn(nEx, self.dim) * 1e-4
        #--------------------------------------------------------------------------#
        # p(ij)
        self.pij = (self.pcond + self.pcond.T) / (2.*nEx)
        #self.pij *= 1 - np.eye(nEx)
        # Descente de Gradient
        loss = []
        for t in xrange(1,self.nIter+1):
            # Matrice des distances 
            normy = np.sum((self.embeddings[t]**2),1)
            normy = np.reshape(normy, (1, nEx))
            distancey = normy + normy.T - 2 * self.embeddings[t].dot(self.embeddings[t].T)
            # q(ij)
            self.qij = (distancey.sum(1).reshape(nEx,1) + (nEx*(nEx-1))) / (1 + distancey)
            #self.qij *= 1 - np.eye(nEx)
            #self.qij = np.sum(self.qij - np.eye(nEx),0) / self.qij
            # Gradient
            tmpgrad = 4 * ((self.pij - self.qij) / (1 + distancey))
            tmpgrad = np.reshape(tmpgrad, (nEx, nEx,1))
            dY = [self.embeddings[t,i] - self.embeddings[t] for i in xrange(nEx)]
            dY = np.array(dY)
            grad = np.sum(tmpgrad * dY, 1)
            # Descente de gradient
            #ymoins1 = self.embedding
            #self.embedding = ymoins1 + self.lr * grad + self.moment * (ymoins1 - ymoins2)
            #ymoins1 = ymoins2
            self.embeddings[t+1] = self.embeddings[t] + self.lr * grad + self.moment * (self.embeddings[t] - self.embeddings[t-1])
            l = np.sum(self.pij * (np.log2(self.pij/self.qij)))
            loss.append(l)
            print t,l

In [7]:
from sklearn import datasets
data = datasets.load_digits()

In [19]:
from sklearn.manifold import TSNE
model = TSNE(n_components=2, random_state=0, n_iter=200)
x = model.fit_transform(data.data) 

plt.figure()
plt.plot(x[:,0][data.target == 0], x[:,1][data.target == 0], 'o', color="blue")
plt.plot(x[:,0][data.target == 1], x[:,1][data.target == 1], 'o', color="red")
plt.plot(x[:,0][data.target == 2], x[:,1][data.target == 2], 'o', color="cyan")
plt.plot(x[:,0][data.target == 3], x[:,1][data.target == 3], 'o', color="magenta")
plt.plot(x[:,0][data.target == 4], x[:,1][data.target == 4], 'o', color="yellow")
plt.plot(x[:,0][data.target == 5], x[:,1][data.target == 5], 'o', color="black")
plt.plot(x[:,0][data.target == 6], x[:,1][data.target == 6], 'o', color="white")
plt.plot(x[:,0][data.target == 7], x[:,1][data.target == 7], 'o', color=(0.5, 0.5, 0))
plt.plot(x[:,0][data.target == 8], x[:,1][data.target == 8], 'o', color=(0,0.5,0.5))
plt.plot(x[:,0][data.target == 9], x[:,1][data.target == 9], 'o', color=(0.5,0,0.5))
plt.show()

In [20]:
model.get_params()

{'early_exaggeration': 4.0,
 'init': 'random',
 'learning_rate': 1000.0,
 'metric': 'euclidean',
 'n_components': 2,
 'n_iter': 200,
 'perplexity': 30.0,
 'random_state': 0,
 'verbose': 0}

In [16]:
model = tSNE(30,100,1e4,0)
model.fit(data.data)

[ 135.20805638  135.24193477   26.39898988 ...,  135.21719547  135.21895758
   74.20709055]
[[  4.91911469   8.38679509  11.59179135 ...,   9.254232     9.27069259
   11.59179135]]
1 -80.5656010474
2 -150.481420611
3 -150.48142063
4 -150.481420634
5 -150.481420599
6 -150.481420417
7 -150.481417756
8 -150.481418004
9 -150.48141834
10 -150.481418868
11 -150.481420127
12 -150.481420236
13 -150.481425639
14 -150.481425649
15 -150.48142566
16 -150.481425673
17 -150.481425688
18 -150.481425705
19 -150.481425726
20 -150.481425752
21 -150.481425788
22 -150.481425839
23 -150.481425925
24 -150.481426111
25 -150.48142705
26 -150.481416642
27 -150.481416656
28 -150.481416671
29 -150.481416686
30 -150.481416701
31 -150.481416718
32 -150.481416735
33 -150.481416754
34 -150.481416777
35 -150.481416809
36 -150.481416911
37 -150.481416916
38 -150.481416919
39 -150.481416916
40 -150.481416864
41 -150.481416928
42 -150.481416516
43 -150.481416529
44 -150.481416542
45 -150.481416555
46 -150.481416569
47 -

In [17]:
t = np.shape(model.embeddings)[0] -1
plt.figure()
plt.plot(model.embeddings[t,:,0][data.target == 0], model.embeddings[t,:,1][data.target == 0], 'o', color="blue")
plt.plot(model.embeddings[t,:,0][data.target == 1], model.embeddings[t,:,1][data.target == 1], 'o', color="red")
plt.plot(model.embeddings[t,:,0][data.target == 2], model.embeddings[t,:,1][data.target == 2], 'o', color="cyan")
plt.plot(model.embeddings[t,:,0][data.target == 3], model.embeddings[t,:,1][data.target == 3], 'o', color="magenta")
plt.plot(model.embeddings[t,:,0][data.target == 4], model.embeddings[t,:,1][data.target == 4], 'o', color="yellow")
plt.plot(model.embeddings[t,:,0][data.target == 5], model.embeddings[t,:,1][data.target == 5], 'o', color="black")
plt.plot(model.embeddings[t,:,0][data.target == 6], model.embeddings[t,:,1][data.target == 6], 'o', color="white")
plt.plot(model.embeddings[t,:,0][data.target == 7], model.embeddings[t,:,1][data.target == 7], 'o', color=(0.5, 0.5, 0))
plt.plot(model.embeddings[t,:,0][data.target == 8], model.embeddings[t,:,1][data.target == 8], 'o', color=(0,0.5,0.5))
plt.plot(model.embeddings[t,:,0][data.target == 9], model.embeddings[t,:,1][data.target == 9], 'o', color=(0.5,0,0.5))
plt.show()

In [176]:
import pickle as pkl
fichier = open("./model.p")
reco = pkl.load(fichier)
fichier.close()

In [178]:
movies = np.zeros((len(reco.q),10))
titles = []
for i,q in enumerate(reco.q.keys()):
    movies[i] = np.reshape(reco.q[q],10,1)
    titles.append(q)

In [180]:
model = tSNE(20,10,1e-3,0.4)
model.fit(movies)

[ 53208.79287149  27978.5242457   63830.81660834 ...,  50168.54788043
  49660.32171641  39566.21720595]
[  94091.36245274    8430.25459954  201165.29926887 ...,   74029.40818452
   70998.92666616   29307.18170129]
[  4.96050963e+05   3.03968269e+02   1.31422839e+07 ...,   1.88630697e+05
   1.52718047e+05   7.32933550e+03]
[  1.07606080e+06   1.07821353e+01   4.58893899e+09 ...,   7.77942855e+04
   2.11222553e+04   1.31130971e+02]
[  6.38815289e-07   3.16139056e+00   3.17928762e+06 ...,   1.98296526e-36
   8.14874245e-04   1.13195949e-99]
[  1.74813038e+05   5.62667614e+00   1.48789267e+02 ...,   1.22448118e+04
   9.20277787e+02   3.20434245e+01]
[  5.16780168e+04   5.38485620e+00   0.00000000e+00 ...,   1.08109920e+03
   6.35681774e+02   2.81515878e+01]
[  2.69388428e+03   6.02597037e+00   6.11872571e-24 ...,   5.36676924e-02
   1.32866277e+02   2.59159929e+01]
[  1.04664727e+01   6.18508951e+00   1.54448444e-03 ...,   8.02304787e+01
   6.93524554e+00   2.59046822e+01]
[ 299.29664194  

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:27: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:27: RuntimeWarning: invalid value encountered in multiply
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:33: RuntimeWarning: invalid value encountered in greater


In [219]:
plt.figure()
plt.plot(model.embeddings[11,:,0],model.embeddings[11,:,1], 'o', color="blue")


#y=[2.56422, 3.77284,3.52623,3.51468,3.02199]
#z=[0.15, 0.3, 0.45, 0.6, 0.75]
#n=[58,651,393,203,123]

#fig, ax = plt.subplots()
#ax.scatter(z, y)

for i, txt in enumerate(titles):
    plt.annotate(txt.decode('latin-1'), model.embeddings[11,i])

plt.show()

In [196]:
+titles[0]

'uAdventures of Pinocchio, The (1996)'

In [130]:
model.embeddings[101]

array([[  3.07592203e+07,   3.77409653e+07],
       [ -4.14262632e+08,   3.50348199e+08],
       [ -2.16739678e+07,  -4.24695982e+07],
       ..., 
       [ -4.52136368e+07,   6.55534182e+07],
       [  2.35250806e+06,  -1.48310620e+07],
       [ -5.12020533e+07,   1.86081529e+08]])

In [137]:
A = np.array([[1,1],[2,2]])
plt.plot(A[:,0], A[:,1], 'o')
plt.show()